In [48]:
from importlib import reload
import numpy as np
import scipy.linalg
import linalg
reload(linalg)
import pywren
reload(pywren)
import math
import itertools
import time
import cloudpickle
import importlib
import sys
from sklearn.datasets import fetch_mldata
from itertools import groupby
from operator import itemgetter
from sklearn.kernel_approximation import RBFSampler, Nystroem
from sklearn import metrics
import sys

In [2]:
def generate_chunked_block_pairs(num_blocks, inner_chunk_size=10, outer_chunk_size=1000):
    all_pairs = list(itertools.product(range(num_blocks), range(num_blocks)))
    sorted_pairs = map(lambda x: tuple(sorted(x)), all_pairs)
    dedup_sorted_pairs = list(set(sorted_pairs))
    print(len(dedup_sorted_pairs))
    return list(chunk(list(chunk(dedup_sorted_pairs, inner_chunk_size)), outer_chunk_size))

def chunk(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [3]:
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [5]:
train_data = np.load("imagenet_train_alexnet_fc7.npz")

In [6]:
X_train = train_data["X_train"]
y_train = train_data["y_train"]
y_train_one_hot = np.eye(1000)[y_train]

In [7]:
X_test = np.load("imagenet_test_features_alexnet_fc7.npy")
y_test = np.load("imagenet_test_labels.npy")

In [9]:
mnist = fetch_mldata('MNIST original', data_home="/home/ubuntu/mnist")
X_train = mnist.data[:60000].astype('float32')/255.0
y_train = mnist.target[:60000].astype('float32')
X_test = mnist.data[60000:].astype('float32')/255.0
y_test = mnist.target[60000:].astype('float32')
y_train_one_hot = np.eye(10)[y_train.astype('int')]
y_test_one_hot = np.eye(10)[y_test.astype('int')]
X_train_sharded = linalg.ShardedMatrix(None, data=X_train, shard_size_0=4096, bucket="imagenet-raw", reshard=True)

/home/ubuntu/pictureweb/pywren_kernels/linalg.py:47: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  assert(data != None)


Sharding matrix..... of shape (60000, 784)


In [11]:
w = np.random.randn(X_train_sharded.shape[0], 10)

In [12]:
%time w_full = linalg.ShardedMatrix(hash_key="w_full", data=w, shard_size_0=4096, bucket="imagenet-raw", n_jobs=8, reshard=True)

[[-0.74587804  0.23294292  1.05410883 ..., -0.48711518  0.78845251
  -0.73061744]
 [-0.55638939 -1.47432333 -1.02358317 ..., -0.63628306 -1.87291958
   0.13081584]
 [ 1.0463728  -1.47212211 -0.14151974 ...,  0.55645552 -2.20675737
   0.79011933]
 ..., 
 [ 0.07007033 -1.18173646  1.19794094 ..., -1.70578322 -0.10774507
  -1.59117371]
 [-1.03784157 -1.8032817   1.27078826 ..., -0.06654361 -0.49236904
   0.69871186]
 [-1.88439331  0.27143937  0.16119008 ...,  0.43041581 -0.64569512
   0.27369092]]
Sharding matrix..... of shape (60000, 10)
CPU times: user 212 ms, sys: 20 ms, total: 232 ms
Wall time: 1.48 s


###### X_train_sharded

In [14]:
# set up pywren
pwex = pywren.default_executor()

In [15]:
num_blocks = int(math.ceil(X_train_sharded.shape[0]/float(X_train_sharded.shard_size_0)))
tasks_per_job = 5
num_jobs = 2500

120


In [17]:
len(chunked_blocks)

1

In [18]:
X_train_sharded.shard_size_0

4096

In [25]:
all_futures = [] 
all_times = []
gamma = 1e-3
lambdav = 1e-2

In [1]:
def compute_kernel_pywren(pwex, X_train_sharded, num_blocks, tasks_per_job, num_jobs):
    chunked_blocks = linalg.generate_chunked_block_pairs(num_blocks, tasks_per_job, num_jobs)
    for c in chunked_blocks:
        %time kernel_futures = pwex.map(lambda x: linalg.compute_rbf_kernel_blockwise(x, X_train_sharded, gamma), c)
        %time pywren.wait(kernel_futures)    
    K_sharded = linalg.compute_rbf_kernel_blockwise([(0,0)], X_train_sharded, gamma)[1]
    return K_sharded

(0, 0)


In [30]:
%time K = linalg.computeRBFGramMatrix(X_train, X_train, gamma=gamma)

CPU times: user 1min 31s, sys: 9.18 s, total: 1min 40s
Wall time: 1min 20s


In [31]:
%time Kw = K.dot(w) 

CPU times: user 12.1 s, sys: 6.25 s, total: 18.4 s
Wall time: 13.7 s


In [49]:
%time Kw_pywren = linalg.pywren_matrix_vector_multiply(pwex, K_sharded, w_full, col_chunk_size=10, row_chunk_size=2750)

15
Num chunks 1
Num total chunks 30
Chunk 0
map submitted now
submit took 0.8258676528930664 seconds
reduce took 1.551887035369873 seconds
CPU times: user 2.22 s, sys: 144 ms, total: 2.36 s
Wall time: 34.3 s


In [51]:
Kw_pywren

array([[ 293.29198925,   75.02265206,  144.6628545 , ...,  122.22586165,
         -62.16654197,  156.41366619],
       [ 290.72771094,   72.5283287 ,  143.16085971, ...,  125.811512  ,
         -62.53056883,  155.42928891],
       [ 286.87118989,   74.2538907 ,  139.71336284, ...,  117.02846291,
         -59.36440179,  159.36465091],
       ..., 
       [ 293.94330255,   87.41353989,  144.59438054, ...,  120.87943979,
         -59.90967562,  168.35489431],
       [ 296.54426493,   90.68159344,  142.29626835, ...,  120.44538073,
         -60.81414137,  166.10904442],
       [ 298.04472853,   92.27840419,  140.15581781, ...,  122.05156521,
         -60.31618221,  167.06428624]])

In [54]:
np.all(np.isclose(Kw_pywren, Kw))

True

In [16]:
K_sharded = linalg.ShardedSymmetricMatrix(K_key, bucket="imagenet-raw")

pywren.linalg/
da82dc87092e9f88e489cb1dd64dbf4c20a5da0e


In [194]:
K_sharded.diag_offset = lambdav

In [19]:
K_sharded.shard_size_1 = 4096

In [20]:
K_sharded.__write_header__()

0

In [217]:

    
def pywren_matrix_vector_multiply_correct(pwex, K, x, col_chunk_size=2):
    num_column_blocks = int(math.ceil(K.shape[0]/float(K.shard_size_0)))
    num_row_blocks = int(math.ceil(K.shape[1]/float(K.shard_size_1)))
    print(num_column_blocks)
    
    all_chunks, num_col_chunks = matrix_chunk(num_column_blocks, num_row_blocks, col_chunk_size)
    print(len(all_chunks))
    all_futures = pwex.map(lambda c: linalg.blocks_matrix_multiply(c, K, x), all_chunks)
    print("Job submitted waiting now")
    pywren.wait(all_futures) 
    print("Done!")
    results = list(map(lambda x: x.result(), all_futures))
    # chunk back into rows 
    # len(chunked_results) == K.shard_size_0 should be true
    chunked_results = list(chunk(results, num_col_chunks))
   
    # sum across columns for each row
    summed_results = list(map(lambda x: np.sum(x, axis=0), chunked_results))
    
    # stack and return
    return np.vstack(summed_results )
    

In [231]:
def pcg_pywren(A,b,pwex, prc=lambda x: x, max_iter=100, tol=1e-3):
    i = 0 
    # starting residual is b
    r = b 
    d = prc(r)
    delta_new = np.linalg.norm(r.T.dot(d))
    delta_0 = delta_new
    print("Delta 0 is {0}".format(delta_0))
    x = np.zeros((A.shape[0], b.shape[1]))
    while (True):
        if (i >= max_iter):
            break
            
        if (delta_new < tol*delta_0):
            break
        # Expensive
        print("Matrix multiply")
        d_sharded = linalg.ShardedMatrix(d, shard_size_0=A.shard_size_0, reshard=True, bucket="imagenet-raw")
        q = pywren_matrix_vector_multiply(pwex, A, d_sharded, col_chunk_size=50, row_chunk_size=2500)
        a = delta_new/np.linalg.norm(d.T.dot(q))
        print(a)
        x = x + a*d
        r = r - a*q
        print("Iter {0}, NORM IS {1}".format(i,np.linalg.norm(r)))
        s = prc(r)
        delta_old = delta_new
        delta_new = np.linalg.norm(r.T.dot(s))
        beta = delta_new/delta_old
        d = s + beta * d
        i = i + 1
        
    return x



In [36]:
def make_smart_precondition(X_train, lambdav, gamma, n_components):
    '''Make a preconditioner kernel ridge regression by using nystroem
       features  '''
    
    print("Computing Nystroem svd")
    nystroem = Nystroem(gamma=gamma, n_components=n_components)
    nystroem.fit(X_train)
    print("Computing Nystroem features")
    X_train_lift = nystroem.transform(X_train)
    print("Computing ZTZ")
    ztz = X_train_lift.T.dot(X_train_lift)
    ztz_reg = ztz + lambdav * np.eye(ztz.shape[0]).astype('float32')
    print("Computing Cholesky")
    L = np.linalg.cholesky(ztz_reg)
    U = scipy.linalg.solve(L, X_train_lift.T)
    print(U.shape)
    def prc(x):
        return (1.0/lambdav)*(x - U.T.dot(U.dot(x)))
    return prc, U

In [37]:
%time prc, U = make_smart_precondition(X_train, lambdav, gamma, n_components=8192)

Computing Nystroem svd
Computing Nystroem features
Computing ZTZ
Computing Cholesky
(8192, 1281167)
CPU times: user 6h 26min 48s, sys: 48min 21s, total: 7h 15min 10s
Wall time: 12min 22s


In [159]:
def pcg(A,b,prc=lambda x: x, max_iter=100, tol=1e-3, reg=1e-3):
    i = 0 
    # starting residual is b
    
    r = b 
    d = prc(r)
    delta_new = np.linalg.norm(r.T.dot(d))
    delta_0 = delta_new
    print("Delta 0 is {0}".format(delta_0))
    diag_indices = np.diag_indices(A.shape[0])
    A[diag_indices] += reg
    x = np.zeros((A.shape[0], b.shape[1]))
    while (True):
        print("Iter {0}".format(i))
        if (i >= max_iter):
            break
            
        if (delta_new < tol*delta_0):
            break
        # Expensive
        q = A.dot(d)
        a = delta_new/np.linalg.norm(d.T.dot(q))
        print(a)
        x = x + a*d
        if (i % 20 == 0):
            r = b - A.dot(x)
        else:
            r = r - a*q
        print("Iter {0}, NORM IS {1}".format(i,np.linalg.norm(r)))
        s = prc(r)
        delta_old = delta_new
        delta_new = np.linalg.norm(r.T.dot(s))
        beta = delta_new/delta_old
        d = s + beta * d
        i = i + 1
    A[diag_indices] -= reg
    return x

In [157]:
%time K = linalg.computeRBFGramMatrix(X_train, X_train, gamma=gamma)

In [160]:
%time model = pcg(K, y_train_one_hot, prc, reg=lambdav, max_iter=1)

In [142]:
%time K_test = linalg.computeRBFGramMatrix(X_test, X_train, gamma=gamma)

CPU times: user 53 s, sys: 14 s, total: 1min 6s
Wall time: 11.6 s


In [143]:
y_test_pred = np.argmax(K_test.dot(model), axis=1)
print(metrics.accuracy_score(y_test, y_test_pred))
y_train_pred = np.argmax(K.dot(model), axis=1)
print(metrics.accuracy_score(y_train, y_train_pred))

0.9804
0.99775


In [233]:
sys.stdout = terminal_stdout
%time model_pywren = pcg_pywren(K_sharded, y_train_one_hot, pwex, prc=prc, max_iter=10)

In [234]:
y_test_pred = np.argmax(K_test.dot(model_pywren), axis=1)
print(metrics.accuracy_score(y_test, y_test_pred))
y_train_pred = np.argmax(K.dot(model_pywren), axis=1)
print(metrics.accuracy_score(y_train, y_train_pred))

In [21]:
%time 

NameError: name 'K_test' is not defined

In [22]:
prc

NameError: name 'prc' is not defined

In [28]:
import dill

In [33]:
import _pickle as cPickle

In [35]:
prc_serialized = cPickle.dumps(prc)

AttributeError: Can't pickle local object 'make_smart_precondition.<locals>.prc'

In [38]:
np.save("U_{gamma}_{lambdav}".format(gamma=gamma, lambdav=lambdav), U)

In [16]:
linalg.make_rbf_hash(X_train_sharded.key, gamma=1e-5)

'da82dc87092e9f88e489cb1dd64dbf4c20a5da0e'